In [ ]:
import glob
import os
import pandas as pd
import numpy as np
import pickle
import datetime
import sys
sys.path.append(r"..\GobyFinder")  # Replace with the actual path to the GobyFinder module
from src import Utils, reports
import matplotlib.pyplot as plt
import datetime

## Indexing the goby collects only (all years)

In [ ]:
def list_goby_collects(op_table_pth, year):
    op_table = pd.read_excel(op_table_pth) # 
    op_table_assmt = op_table[op_table.GOBY == 1]
    df_goby_sites = op_table_assmt[op_table_assmt["COLLECT_ID"].str.contains(f"{year}")]
    return df_goby_sites['COLLECT_ID']
## Generate metadata from all unpacked images
def usable_goby_metadata(aluim, op_table_pth, year):
    goby_collect_list = list_goby_collects(op_table_pth, year)
    print(len(goby_collect_list), "goby collects from", year)
    meta_df = aluim
    print("original metadata", meta_df.shape)
    meta_potentially_usable = meta_df[(meta_df['Metadata Thresholds'] == "Pass")]
    meta_usable_rf = meta_potentially_usable[(meta_potentially_usable['Usability Random Forest'] == "Pass")]
    meta_usable = meta_df[(meta_df['Usability'] == "Usable")]
    print("portion usable out of potential", meta_usable_rf.shape[0]/meta_potentially_usable.shape[0])
    meta_usable_goby = meta_usable[meta_usable.collect_id.isin(goby_collect_list)]
    #Checks
    orig_shape = meta_usable_goby.shape[0]
    print("Goby collects metadata", orig_shape)
    drop_fnd = meta_usable_goby.drop_duplicates(subset="Filename").shape[0]
    print(orig_shape - drop_fnd, "duplicate filenames")
    drop_fnn = meta_usable_goby.dropna(subset="Filename").shape[0]
    print(orig_shape - drop_fnn, "missing filenames")
    drop_ipd = meta_usable_goby.drop_duplicates(subset="image_path").shape[0]
    print(orig_shape - drop_ipd, "duplicate image paths")
    drop_tsn = meta_usable_goby.dropna(subset="Time_s").shape[0]
    print(orig_shape - drop_tsn, "missing timestamps")
    drop_dbn = meta_usable_goby.dropna(subset="DistanceToBottom_m").shape[0]
    print(orig_shape - drop_dbn, "missing altitudes")
    # dropping nan in DistanceToBottom_m
    meta_usable_goby_dbn = meta_usable_goby.dropna(subset="DistanceToBottom_m")
    print("final table", meta_usable_goby_dbn.shape)
    ## median 
    medians = meta_usable_goby_dbn.groupby('collect_id')['Time_s'].apply(lambda x: x.diff()).values
    print("Average median time interval", np.nanmedian(medians))
    return meta_usable_goby_dbn


## 2024

In [ ]:
op_table_pth = r"Z:\__AdvancedTechnologyBackup\01_DerivedProducts\Database\OP_TABLE.xlsx"
t = datetime.datetime.now()
Ymmdd = f"{t.year:02d}{t.month:02d}{t.day:02d}"
print("YYYYmmdd:", Ymmdd)
metadata = pd.read_pickle(r"Z:\__AdvancedTechnologyBackup\01_DerivedProducts\Database\02__MetadataCombined\all_unpacked_images_metadata.pickle")

In [ ]:
year = 2024
aluim = metadata[metadata.year == str(year)].copy()

'''
83 goby collects from 2024
original metadata (200802, 169)
portion usable out of potential 0.6926015422337515
Goby collects metadata 108082
0 duplicate filenames
0 missing filenames
0 duplicate image paths
0 missing timestamps
2522 missing altitudes
final table (105560, 169)
Average median time interval 5.0
'''
meta_2024_usable_gobydd = usable_goby_metadata(aluim, op_table_pth, 2024)
# meta_2024_usable_gobydd.to_csv(r"Z:\Proj_Yolo\2024_Yolo_update\02_2024_Assessment\meta_2024_usable_goby.csv")

In [ ]:
meta_2024_usable_gobydd[]

In [ ]:
img_lst_2024 = list(meta_2024_usable_gobydd.image_path)
filename = r"Z:\Proj_Yolo\2024_Yolo_update\02_2024_Assessment\img_lst_2024.csv"
# general.write_list_txt(filename, img_lst_2024)

## 2023, earlier

In [ ]:
year = 2023
aluim = metadata[metadata.year == str(year)].copy()
'''
94 goby collects from 2023
original metadata (278703, 158)
portion usable out of potential 0.5728057392668984
Goby collects metadata 109789
0 duplicate filenames
0 missing filenames
0 duplicate image paths
0 missing timestamps
6112 missing altitudes
final table (103677, 158)
Average median time interval 4.999852895736694
'''
meta_2023_usable_gobydd = usable_goby_metadata(aluim, op_table_pth, 2023)
# meta_2023_usable_gobydd.to_csv(r"Z:\Proj_Yolo\2023_Yolo_update\02_2023_Assessment\meta_2023_usable_goby2.csv")

### 2022

In [ ]:
year = 2022
aluim = metadata[metadata.year == str(year)].copy()

'''
original metadata (241828, 145)
Drop unusable (113769, 145)
Goby collects metadata (107651, 145)
drop filename nan (107651, 145)
drop filename dup (107651, 145)
drop Time_s nan (107651, 145)
drop DistanceToBottom_m nan (100085, 145)
drop image_path dup (107651, 145)
final table (100085, 145)
Average median time interval 4.9998345375061035
'''
meta_2022_usable_gobydd = usable_goby_metadata(aluim, op_table_pth, 2022)

### 2021

In [ ]:
aluim = r"Z:\Database\02__MetadataCombined\all_unpacked_images_metadata_2021.csv"
'''
83 goby collects from 2021
original metadata (275149, 156)
portion usable out of potential 0.4529760084512524
Goby collects metadata 86275
0 duplicate filenames
0 missing filenames
0 duplicate image paths
0 missing timestamps
556 missing altitudes
final table (85719, 156)
Average median time interval 4.019714117050171
'''
meta_2021_usable_gobydd = usable_goby_metadata(aluim,op_table_pth, 2021)

### 2020

In [ ]:
aluim = r"Z:\Database\02__MetadataCombined\all_unpacked_images_metadata_2020.csv"
'''
original metadata (112808, 123)
Drop unusable (51346, 123)
Goby collects metadata (51346, 123)
drop filename nan (51346, 123)
drop filename dup (51346, 123)
drop Time_s nan (51346, 123)
drop DistanceToBottom_m nan (50380, 123)
drop image_path dup (51346, 123)
final table (50380, 123)
Average median time interval 4.0
'''
meta_2020_usable_gobydd = usable_goby_metadata(aluim, op_table_pth, 2020)

In [ ]:
# meta_2020_usable_gobydd.to_csv(os.path.join(r"Z:\Proj_Yolo\2020_Yolo_update\02_2020_Assessment\2020_Assessment_11-2", f"meta_2020_usable_goby_{Ymmdd}.csv"))

## April - May shallow study

In [ ]:
columns2 = ['Time_s', 'Filename', 'Latitude', 'Longitude', 'DepthFromSurface_m', 'DistanceToBottom_m', 'TotalWaterColumn_m','Speed_kn', 'Time_UTC', 'DataSource',
            'CollectID', 'BagFile', 'CameraRoll','Usability Random Forest', 'Usability', 'collect_id', 'PointCloudStandardDeviation', 'PointCloudElevation_m', 'year','month', 'day', 'time', 'imw', 'imh', 'image_path']
# all_unpacked = pd.read_pickle(r"Z:\Database\02__MetadataCombined\all_unpacked_images_metadata.pickle")[columns2]
# au_april_may = all_unpacked[(all_unpacked.Usability=="Usable") & (all_unpacked.month.isin([4,5])) & (all_unpacked.DepthFromSurface_m<30)]
# au_april_may.to_csv(r"inference\april-may-shallow\april-may-shallow-metadata.csv")
# au_april_may_lst = list(au_april_may.image_path)
# filename = r"inference\april-may-shallow\april-may-shallow-metadata_lst.csv"
# general.write_list_txt(filename, au_april_may_lst)
# au_april_may

## Balanced Test Assessment

In [ ]:
def balanced_set(runs, df_20_23_alln):
    dev_all = pd.DataFrame()    
    for run in runs:
        rt = pd.read_csv(f"datasets\\AUV_datasets\\Runs\\{run}\\train_df.csv", index_col=0)
        rf = pd.read_csv(f"datasets\\AUV_datasets\\Runs\\{run}\\valid_df.csv", index_col=0)
        dev_all = pd.concat([dev_all, rt, rf])
    df = df_20_23_alln[~df_20_23_alln.image_id.isin(dev_all.Filename)]
    min_year_count = df.year.value_counts().min()
    balanced_test_df = df.groupby('year', group_keys=False).apply(lambda x: x.sample(min_year_count, random_state=123))
    return balanced_test_df.reset_index()

def copy_balanced_test(balanced_test_df, year):
    balanced_test_year_df = balanced_test_df[balanced_test_df.year == year]
    l = len(balanced_test_year_df)
    for idx, row in balanced_test_year_df.iterrows():
        index, image_path, Filename, image_id, bbox_path, mer_path, year, n_fish = row
        dest1 = f"datasets\\AUV_datasets\\test_sets\\balanced_test\\{year}\\images"
        dest2 = f"datasets\\AUV_datasets\\test_sets\\balanced_test\\{year}\\labels"
        if not os.path.exists(dest1): os.makedirs(dest1)
        if not os.path.exists(dest2): os.makedirs(dest2)
        dest_image_path = os.path.join(dest1, os.path.basename(image_path))
        dest_bbox_path = os.path.join(dest2, os.path.basename(bbox_path))
        if not os.path.exists(dest_image_path):
            shutil.copy2(image_path, dest_image_path)
            print(idx, "/", l, end = "   \r")
        if not os.path.exists(dest_bbox_path):
            shutil.copy2(bbox_path, dest_bbox_path)
            print(idx, "/", l, end = "   \r")


In [ ]:
df_20_23_alln = pd.read_csv(r"datasets\AUV_datasets\All_Goby_imgs_lbls_pths_n_fish.csv", index_col=0)
# A test set that does not include any of the previous development sets in given runs
balanced_test_df = balanced_set(runs=["run12"], df_20_23_alln= df_20_23_alln)

# Check if the balanced_test_df dataframe has the expected structure and values
expected_image_ids = ["PI_1598268549_144_Iver3069", "PI_1663618220_651_Iver3069", "PI_1660504715_790_Iver3098"]
assert all(balanced_test_df.loc[idx, "image_id"] == expected_id for idx, expected_id in zip([10, 2355, 2364], expected_image_ids)), "Dataframe does not match expected values"

# Check if the dataframe has the expected number of rows and columns
expected_shape = (3336, 8)
assert balanced_test_df.shape == expected_shape, f"Expected shape {expected_shape}, but got {balanced_test_df.shape}"
print(balanced_test_df.shape)
'''year
2020    824
2021    824
2022    824
2023    824'''
print(balanced_test_df.year.value_counts())

In [ ]:
import shutil
copy_balanced_test(balanced_test_df, year=2023)

In [ ]:
annotated_imgs_metadata = pd.read_csv(r"Z:\Database\02__MetadataCombined\all_unpacked_images_metadata_annotated.csv", index_col=0, low_memory=False)
balanced_test_metadata_df = annotated_imgs_metadata[annotated_imgs_metadata.Filename.isin(balanced_test_df.image_id)]
assert balanced_test_metadata_df.shape[0] == balanced_test_df.shape[0]
columns2 = ['Time_s', 'Filename', 'Latitude', 'Longitude', 'DepthFromSurface_m', 'DistanceToBottom_m', 'TotalWaterColumn_m','Speed_kn', 'Time_UTC', 'DataSource', 'InvalidTelnetLines',
       'CollectID', 'BagFile', 'CameraRoll','Usability Random Forest', 'Usability', 'collect_id', 'PointCloudStandardDeviation', 'PointCloudElevation_m', 'year','month', 'day', 'time', 'imw', 'imh', 'image_path']
balanced_test_metadata_df = balanced_test_metadata_df[columns2]
# balanced_test_metadata_df.to_csv(r"C:\Users\ageglio\ageglio-1\gobyfinder_yolov8\inference\Balanced_test_init_assmt\balanced_test_metadata_df.csv")
# filename = r"C:\Users\ageglio\ageglio-1\gobyfinder_yolov8\inference\Balanced_test_init_assmt\balanced_test_metadata_lst.csv"
# general.write_list_txt(filename, balanced_test_metadata_df.image_path)


### After running YOLO_predict with labels

In [ ]:
conf_thresh = 0.43
df_pred = pd.read_csv(r"test_runs\detect\balanced_test_init_assmt\balanced_test_init_assmt_predictions.csv", index_col=0)
df_lbls = pd.read_csv(r"test_runs\detect\balanced_test_init_assmt\balanced_test_init_assmt_Labels.csv", index_col=0)
fn_df = reports().return_fn_df(df_lbls, df_pred, iou_tp = 0.5, conf_thresh = conf_thresh)
scores_df = reports().scores_df(df_lbls, df_pred, iou_tp = 0.5)
## FP dataframe
fp_scores = scores_df[(scores_df.fp==1) & (scores_df.conf>=conf_thresh)]
fp_scores = fp_scores.rename(columns = {"x_p":"x", "y_p":"y", "w_p":"w", "h_p":"h"})
fp_scores.to_csv(r"inference\Balanced_test_init_assmt\balanced_fp_scores.csv")
## TP dataframe
tp_scores = scores_df[(scores_df.tp==1) & (scores_df.conf>=conf_thresh)]
tp_scores = tp_scores.rename(columns = {"x_p":"x", "y_p":"y", "w_p":"w", "h_p":"h"})
tp_scores.to_csv(r"inference\Balanced_test_init_assmt\balanced_tp_scores.csv")
## FN dataframe
## Detections have to be pre-filtered by confidence threshold for analysis of false negatives
fn_df = fn_df[(fn_df.fn==1)]
fn_df = fn_df.rename(columns = {"x_l":"x", "y_l":"y", "w_l":"w", "h_l":"h"})
fn_df.to_csv(r"inference\Balanced_test_init_assmt\balanced_fn_scores.csv")

In [ ]:
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

def freq_dist_month(df, ax, label, color, position):
    unix_times = df['Time_s'].values
    dates = [datetime.utcfromtimestamp(ts) for ts in unix_times]
    date_df = pd.DataFrame(dates, columns=['date'])
    date_df['month'] = date_df['date'].dt.to_period('M')
    monthly_counts = date_df['month'].value_counts().sort_index()
    monthly_counts.index = monthly_counts.index.to_timestamp()
    ax.bar(monthly_counts.index + pd.DateOffset(days=position), monthly_counts.values, width=20, label=label, color=color)
    for date, count in monthly_counts.items():
        if date.month == 9:
            ax.annotate(f'{date.year}', xy=(date, count), xytext=(0, 5), textcoords='offset points', ha='center', fontsize=8, color=color)

fig, ax = plt.subplots(figsize=(15, 6))
freq_dist_month(annotated_imgs_metadata, ax, label='Annotated Images', color='blue', position=0)
freq_dist_month(balanced_test_metadata_df, ax, label='Balanced Test', color='orange', position=0)

# Add vertical dotted lines for January
years = annotated_imgs_metadata['Time_s'].apply(lambda x: datetime.utcfromtimestamp(x).year).unique()
for year in years:
    jan = datetime(year, 1, 1)
    ax.axvline(jan, color='gray', linestyle='dotted')

ax.set_xlabel('Month')
ax.set_ylabel('Number of Samples')
ax.set_title('Frequency Distribution of Samples per Month')
ax.xaxis.set_major_locator(mdates.MonthLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))
ax.set_yscale('log')
plt.xticks(rotation=45)
plt.legend()

plt.show()